In [83]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random



In [84]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!', '.', ':', ',', 'I', 'am', 'feel', 'like', 'having', 'is']
data_file = open('fresh_intents.json').read()
intents = json.loads(data_file)



In [85]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # print(documents)
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
# print (len(documents), "documents")
# print('Documents', documents)
# classes = intents
# print (len(classes), "classes", classes)
# words = all words, vocabulary
# print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [86]:
print(documents)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Hay'], 'greeting'), (['Greetings'], 'greeting'), (['Howdy'], 'greeting'), (['bonjour'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['adios'], 'goodbye'), (['so', 'long'], 'goodbye'), (['ciao'], 'goodbye'), (['adieu'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', 'is', 'helpful'], 'thanks'), (['Thanks', 'for', 'the', 'help'], 'thanks'), (['Thank', 'you', 'for', 'the', 'assistance'], 'thanks'), (['Much', 'obliged'], 'thanks'), (['Thank', 'you', 'for', 'the', 'aid'], 'thanks'), (['Thank', 'you', 'for', 'the', 'support'], 'thanks'), (['Thank', 'you', 'for', 'the', 'service'], 'thanks'), (['Who', 'are', 'you', '?'], 'about'), (['What', 'are', 'you', '?'], 'about'), (['Can', 'you', 'tell', 'me', 'who', 'you', 'are', '?'], 'about'), (['I', 'would', 'like', 'to

In [87]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print(pattern_words)
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
#     print(pattern_words)
list_pattern = pattern_words
print(list_pattern)
#     print(bag)
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")



['Hi']
['Hey']
['Is', 'anyone', 'there', '?']
['Hello']
['Hay']
['Greetings']
['Howdy']
['bonjour']
['Bye']
['See', 'you', 'later']
['Goodbye']
['bye', 'bye']
['adios']
['so', 'long']
['ciao']
['adieu']
['Thanks']
['Thank', 'you']
['That', 'is', 'helpful']
['Thanks', 'for', 'the', 'help']
['Thank', 'you', 'for', 'the', 'assistance']
['Much', 'obliged']
['Thank', 'you', 'for', 'the', 'aid']
['Thank', 'you', 'for', 'the', 'support']
['Thank', 'you', 'for', 'the', 'service']
['Who', 'are', 'you', '?']
['What', 'are', 'you', '?']
['Can', 'you', 'tell', 'me', 'who', 'you', 'are', '?']
['I', 'would', 'like', 'to', 'talk', 'about', 'my', 'feelings']
['Is', 'this', 'a', 'safe', 'space', 'to', 'talk', '?']
['I', 'am', 'feeling', 'upset']
['I', 'am', 'not', 'feeling', 'well', 'today']
['Yes', ',', 'I', 'want', 'to', 'talk', '.']
['Sure']
['Of', 'course']
['yes']
['years', 'old']
['I', 'decline']
['My', 'address', 'is']
['My', 'zip', 'code', 'is']
['The', 'city', 'I', 'live', 'in']
['I', 'live', 

<ipython-input-87-7d33aee91bc2>:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [88]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
78/78 [==============================] - 0s 2ms/step - loss: 2.0762 - accuracy: 0.3342
Epoch 2/200
78/78 [==============================] - 0s 2ms/step - loss: 1.7436 - accuracy: 0.3964
Epoch 3/200
78/78 [==============================] - 0s 2ms/step - loss: 1.7081 - accuracy: 0.4301
Epoch 4/200
78/78 [==============================] - 0s 1ms/step - loss: 1.6143 - accuracy: 0.4404
Epoch 5/200
78/78 [==============================] - 0s 1ms/step - loss: 1.5220 - accuracy: 0.4637
Epoch 6/200
78/78 [==============================] - 0s 1ms/step - loss: 1.4370 - accuracy: 0.4948
Epoch 7/200
78/78 [==============================] - 0s 1ms/step - loss: 1.3974 - accuracy: 0.5155
Epoch 8/200
78/78 [==============================] - 0s 2ms/step - loss: 1.2617 - accuracy: 0.5674
Epoch 9/200
78/78 [==============================] - 0s 1ms/step - loss: 1.1122 - accuracy: 0.6528
Epoch 10/200
78/78 [==============================] - 0s 1ms/step - loss: 0.9504 - accuracy: 0.6710
Epoch 11/

78/78 [==============================] - 0s 1ms/step - loss: 0.0772 - accuracy: 0.9741
Epoch 84/200
78/78 [==============================] - 0s 2ms/step - loss: 0.0586 - accuracy: 0.9793
Epoch 85/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0935 - accuracy: 0.9715
Epoch 86/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0755 - accuracy: 0.9715: 0s - loss: 0.0757 - accuracy: 0.97
Epoch 87/200
78/78 [==============================] - 0s 2ms/step - loss: 0.0846 - accuracy: 0.9689
Epoch 88/200
78/78 [==============================] - 0s 2ms/step - loss: 0.1042 - accuracy: 0.9767
Epoch 89/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0599 - accuracy: 0.9767
Epoch 90/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0553 - accuracy: 0.9741
Epoch 91/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0750 - accuracy: 0.9741
Epoch 92/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0

78/78 [==============================] - 0s 1ms/step - loss: 0.0334 - accuracy: 0.9845
Epoch 165/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0363 - accuracy: 0.9793
Epoch 166/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0295 - accuracy: 0.9870
Epoch 167/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0422 - accuracy: 0.9819
Epoch 168/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0512 - accuracy: 0.9767
Epoch 169/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0373 - accuracy: 0.9870
Epoch 170/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0723 - accuracy: 0.9741
Epoch 171/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0284 - accuracy: 0.9870
Epoch 172/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0539 - accuracy: 0.9845
Epoch 173/200
78/78 [==============================] - 0s 1ms/step - loss: 0.0702 - accuracy: 0.9741
Epoc

In [89]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
import pickle
import numpy as np
# import chatbot
import re
regex = re.compile('[^a-zA-Z]')
from keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('fresh_intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
import emoji
#Creating GUI with tkinter
import tkinter
from tkinter import *
from googlesearch import search
from itertools import chain
res3 = None
res4 = None


In [90]:
# documents = chatbot.documents
# ignore_words = chatbot.ignore_words
ignore_words = [lemmatizer.lemmatize(word.lower()) for word in ignore_words]
ignore_char = ['?', '!', '.', ':', ',']

# print(documents)
docs = [documents[j][0] for j in range(len(documents))]
# print(docs)

for i in ignore_char:
    for s in docs:
        if i in s:
#             print(i, "in", s)
#             print(i in s)
            s.remove(i)
#             print(s)
print(docs)
#             docs = [s.remove(i) for s in docs if i in s]
# print(docs)
# for i in docs:
# #     print(i)
#     for j in ignore_char:
#         if j in docs:
#             docs = [s.replace(j, "") for s in i]
#             print(docs)

# for i in ignore_char:
#     docs = [s.replace(i, "") for s in docs]

[['Hi'], ['Hey'], ['Is', 'anyone', 'there'], ['Hello'], ['Hay'], ['Greetings'], ['Howdy'], ['bonjour'], ['Bye'], ['See', 'you', 'later'], ['Goodbye'], ['bye', 'bye'], ['adios'], ['so', 'long'], ['ciao'], ['adieu'], ['Thanks'], ['Thank', 'you'], ['That', 'is', 'helpful'], ['Thanks', 'for', 'the', 'help'], ['Thank', 'you', 'for', 'the', 'assistance'], ['Much', 'obliged'], ['Thank', 'you', 'for', 'the', 'aid'], ['Thank', 'you', 'for', 'the', 'support'], ['Thank', 'you', 'for', 'the', 'service'], ['Who', 'are', 'you'], ['What', 'are', 'you'], ['Can', 'you', 'tell', 'me', 'who', 'you', 'are'], ['I', 'would', 'like', 'to', 'talk', 'about', 'my', 'feelings'], ['Is', 'this', 'a', 'safe', 'space', 'to', 'talk'], ['I', 'am', 'feeling', 'upset'], ['I', 'am', 'not', 'feeling', 'well', 'today'], ['Yes', 'I', 'want', 'to', 'talk'], ['Sure'], ['Of', 'course'], ['yes'], ['years', 'old'], ['I', 'decline'], ['My', 'address', 'is'], ['My', 'zip', 'code', 'is'], ['The', 'city', 'I', 'live', 'in'], ['I', '

In [91]:
doc_lst = []
for i in range(len(docs)):
    doc_lst.append(" ".join(docs[i]))
doc_lst = [lemmatizer.lemmatize(word.lower()) for word in doc_lst]

# print(doc_lst)

# for i in ignore_char:
#     doc_lst = [s.replace(i, "") for s in doc_lst]

# for i in ignore_char:
#     print(i)
#     for doc in doc_lst:
#         if i in doc:
#             doc.replace("i", "")
#             new_lst.append(doc)
# print(new_lst)


# new_list = [s.replace("e", "") for s in list_of_strings]

print(doc_lst)

['hi', 'hey', 'is anyone there', 'hello', 'hay', 'greeting', 'howdy', 'bonjour', 'bye', 'see you later', 'goodbye', 'bye bye', 'adios', 'so long', 'ciao', 'adieu', 'thanks', 'thank you', 'that is helpful', 'thanks for the help', 'thank you for the assistance', 'much obliged', 'thank you for the aid', 'thank you for the support', 'thank you for the service', 'who are you', 'what are you', 'can you tell me who you are', 'i would like to talk about my feelings', 'is this a safe space to talk', 'i am feeling upset', 'i am not feeling well today', 'yes i want to talk', 'sure', 'of course', 'yes', 'years old', 'i decline', 'my address is', 'my zip code is', 'the city i live in', 'i live near', 'the city i live near', 'student', 'teacher', 'nurse', 'doctor', 'banker', 'lawyer', 'fireman', 'policeman', 'logger', 'cashier', 'manager', 'lab technician', 'scientist', 'developer', 'professor', 'data scientist', 'chef', 'cook', 'sous chef', 'dish washer', 'administrator', 'ceo', 'cfo', 'coo', 'vice

In [92]:
print(ignore_words)

['?', '!', '.', ':', ',', 'i', 'am', 'feel', 'like', 'having', 'is']


In [93]:
print(doc_lst)

for doc in doc_lst:
#     for i in doc:
    print(doc)

['hi', 'hey', 'is anyone there', 'hello', 'hay', 'greeting', 'howdy', 'bonjour', 'bye', 'see you later', 'goodbye', 'bye bye', 'adios', 'so long', 'ciao', 'adieu', 'thanks', 'thank you', 'that is helpful', 'thanks for the help', 'thank you for the assistance', 'much obliged', 'thank you for the aid', 'thank you for the support', 'thank you for the service', 'who are you', 'what are you', 'can you tell me who you are', 'i would like to talk about my feelings', 'is this a safe space to talk', 'i am feeling upset', 'i am not feeling well today', 'yes i want to talk', 'sure', 'of course', 'yes', 'years old', 'i decline', 'my address is', 'my zip code is', 'the city i live in', 'i live near', 'the city i live near', 'student', 'teacher', 'nurse', 'doctor', 'banker', 'lawyer', 'fireman', 'policeman', 'logger', 'cashier', 'manager', 'lab technician', 'scientist', 'developer', 'professor', 'data scientist', 'chef', 'cook', 'sous chef', 'dish washer', 'administrator', 'ceo', 'cfo', 'coo', 'vice

In [94]:
msg = "Yes, I want to talk?"

for i in ignore_char:
    msg = msg.replace(i,"").lower()
# print(msg.lower())

print(msg in doc_lst)

True


In [95]:
#Tokenizes and lemmatizes the sentences
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print(sentence_words)
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
#Tokenizes pattern and returns an array of the bag of words
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [96]:
#Function filters out bot responses below a 25% threshold. Returns most likely
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [97]:
# Loading the glove vector from the system
def load_glove_model(File):
    # print("Loading Glove Model")
    glove_model = {}
    with open(File, 'r', encoding="utf8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    # print(f"{len(glove_model)} words loaded!")
    return glove_model
glove_vectors = load_glove_model("glove.6B.50d.txt")


In [98]:
# creating the word vectors from the input
def get_sentence_vector(sentence):
    sent_array = []
    for word in sentence.split():
        temp = word.strip()
        # print("Temp", temp)
        temp = regex.sub('', temp)
        # print("Temp", temp)
        temp = temp.lower()
        # print("Temp", temp)
        sent_array.append(glove_vectors[temp])
        # print("sent_array", sent_array)
    return(np.mean(sent_array, axis=0))

In [99]:
# Semantic analysis
def semantic_search(user_input, intents_json):    
    sim_score = []
    for i in range(len(documents)):
        # print(corpus)
        doc1 = user_input
        # print(doc1)
        doc1_vector = get_sentence_vector(doc1)
        doc2 = [doc for doc in documents[i][0] if doc not in ignore_words]
        # print(doc2)
        doc2 = " ".join(doc2)
        # print(doc2)
        doc2_vector = get_sentence_vector(doc2)    
        sim_lst = cosine_similarity([doc1_vector, doc2_vector])
        sim_score.append(sim_lst[1][0])
    tag = documents[sim_score.index(max(sim_score))][1]
    print(tag)
    # print(max(sim_score))
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result, tag


In [100]:
# Assess the severity of mental illness and provides response accordingly
def sentiment_analysis(user_input):
    scores = sid.polarity_scores(user_input)
    return scores['compound']
#     if scores['compound'] <-0.2:
#         return 'Psychiatric consultation'
#     elif (-0.2 <= scores['compound'] <= 0.2):
#         return 'send youtube video'
#     else:
#         return 'you are ok'


In [101]:
#Based off intents file tags generates a response
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    print(tag)
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result, tag

In [102]:
#Handles the bots responses based off the input and intents file and semantic search
def chatbot_response(msg):
    global res3, res4
    print("Message is", msg)
    for i in ignore_char:
        msg = msg.replace(i,"").lower()

# print(doc_lst)
    if msg in doc_lst:
        print(msg in doc_lst)
        ints = predict_class(msg, model)
        print(ints)
        res1 = getResponse(ints, intents)
        if res1[1] in ('Depression', 'psychosis'):
            res3 = sentiment_analysis(msg)
        return res1[0], res3
    else:
        print(msg in doc_lst)
        res2 = semantic_search(msg, intents)
        if res2[1] in ('Depression', 'psychosis'):
            res4 = sentiment_analysis(msg)
        return res2[0], res4


In [103]:
def give_url(message):
    #We can change the numbers later this is just for the test
    sentiment_analysis_test = chatbot_response(msg)[1]
    if sentiment_analysis_test < -0.2:
        try:
                         #Google Search query results as a Python List of URLs
                         query = 'psychiatrists near ' + user_zipcode[0]
                         #Right now it gathers three results from google and puts them into a list. num is the number of results it will find. stop is how many it will put into the list. pause is how many times it will search(I think).
                         search_result_list = list(search(query, tld="co.in", num=3, stop=3, pause=1))
                         #prints out the url if it contains psychology today. This can be changed.
                         for url in search_result_list:
                            if url.startswith('https://www.psychologytoday.com'):
                                print(url)
                                return str("Here are a list of psychiatrists in your area: ", url)
        except:
                         return str("Something went wrong with your video search. Please check your internet connection and try again.")
    elif -0.2 <= sentiment_analysis_test <= 0.2:
        try:
                         #Google Search query results as a Python List of URLs
                         query = 'meditation youtube videos'
                         #Right now it gathers three results from google and puts them into a list. num is the number of results it will find. stop is how many it will put into the list. pause is how many times it will search(I think).
                         search_result_list = list(search(query, tld="co.in", num=3, stop=3, pause=1))
                         #prints out the url if it contains psychology today. This can be changed.
                         for url in search_result_list:
                                print(url)
                                return str("Here are some calming Youtube videos for you: ", url)
        except:
                         return str("Something went wrong with your video search. Please check your internet connection and try again.")
    else:
        return "You are fine take a walk and clear your head"

In [105]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    
#     stored_response = "May I know where you live? You can give me your zip code just say \"My zip code is\" "
#     name_response = name_checker(msg)
#     age_response = age_checker(msg)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        
        #We will need to consider these two lines when responding. 
        #Could modify the response from the JSON file here. This would enable us to add personal name and other details that the user enters.
        res = chatbot_response(msg)[0]
        try:
            #If the user enters the name then the bot will respond with that name
            if res.startswith("Hello"):
                ChatLog.insert(END, "Bot: " + res.format(username = name_response) + '\n\n')
            #If the user enters the zip code than the zipsearch function is activated and a nearby psychatrist will be google searched and a url is returned
            #elif len(msg) == 5 and msg.isdigit() and msg.startswith("0"):
            elif msg.startswith("My zip code is"):
                msg = msg.strip("My zip code is")
                custom_response = zipsearch(msg)
                if custom_response != "Thank you for answering.":
                    ChatLog.insert(END, custom_response + '\n\n')
                else:
                    ChatLog.insert(END, custom_response + '\n\n')
                    ChatLog.insert(END, "Bot: " + res + '\n\n')
            #All other inputs will be handled here
            elif msg == "I decline":
                ChatLog.insert(END, "Bot: " + res + '\n\n')
                ChatLog.insert(END, "Bot: " + stored_response + '\n\n')
            else:
                ChatLog.insert(END, "Bot: " + res + '\n\n')
                
                #This will still run even if the user doesnt activate the custom response so have to put a try/except block here.
        except TypeError:
            pass
        
        
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

    
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


Message is It is the fear of failure, coupled with no urge to be productive. It is wanting friends but hating socializing. It is wanting to be alone but not wanting to be lonely. It is feeling everything at once, yet being crippling numb.
False
Depression
Message is Is this a safe space to talk?
True
['is', 'this', 'a', 'safe', 'space', 'to', 'talk']
[{'intent': 'feelings - questions', 'probability': '0.99968874'}]
feelings - questions
Message is I have a deep sense of sorrow
False
Depression


In [79]:
doc_lst = ['who are you ?', 'what are you ?', 'can you tell me who you are ?', 'i would like to talk about my feelings.', 'is this a safe space to talk ?', 'i am feeling upset', 'i am not feeling well today', 'yes , i want to talk .']

In [80]:
print(doc_lst)

['who are you ?', 'what are you ?', 'can you tell me who you are ?', 'i would like to talk about my feelings.', 'is this a safe space to talk ?', 'i am feeling upset', 'i am not feeling well today', 'yes , i want to talk .']


In [81]:
ignore_char = ['?', '!', '.', ':', ',']

In [82]:
for i in ignore_char:
    doc_lst = [s.replace(i, "") for s in doc_lst]

In [152]:
print(doc_lst)

['who are you ', 'what are you ', 'can you tell me who you are ', 'i would like to talk about my feelings', 'is this a safe space to talk ', 'i am feeling upset', 'i am not feeling well today', 'yes  i want to talk ']


In [114]:
from googlesearch import search

In [115]:
query = "psychiatrists near" + " 01609"

In [116]:
query

'psychiatrists near 01609'

In [121]:
for j in search(query):
    print(j)


https://www.psychologytoday.com/us/psychiatrists/ma/worcester-county
https://health.usnews.com/doctors/psychiatrists/massachusetts/worcester
https://www.healthgrades.com/psychiatry-directory/ma-massachusetts/worcester
https://doctor.webmd.com/find-a-doctor/specialty/psychiatry/massachusetts/worcester
https://www.yelp.com/search?cflt=psychiatrists&find_loc=Worcester%2C+MA
https://threebestrated.com/psychiatrists-in-worcester-ma
https://www.zocdoc.com/psychiatrists/worcester-217709pm
http://www.prioritypsychiatry.com/
https://www.vitals.com/psychiatrists/ma/worcester
https://www.ummhealth.org/umass-memorial-medical-center/services-treatments/psychiatry/services-we-provide/outpatient-psychiatry
https://www.umassmed.edu/psychiatry/rappid/resources-for-women/mental-healthcare-providers/
